##### AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Programu Operacyjnego Polska Cyfrowa na lata 2014-2020
<hr>


In [ ]:
#@title
%%html
<iframe src="https://www.polskacyfrowa.gov.pl/media/48246/FE_POPC_poziom_pl-1_rgb.jpg" width="800"></iframe>


# Uczenie głębokie

Szymon Zaporowski, Politechnika Gdańska, Wydział ETI, Katedra Systemów Multimedialnych

**Wykład 8:** Rekurencyjne Sieci Neuronowe

**Przykład (1):** Modelowanie sekwencji z użyciem sieci reurencyjnych


Poniższy przykład będzie prosta ilustracją w jaki sposób z użyciem różnych wariantów sieci rekurencyjnych można prognować szeregi czasowe.
Wykorzystamy naiwny przykład z użyciem jednowymiarowych danych tworzących pewien szereg.

Zaimportujmy niezbędne biblioteki:

In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import GRU

Zdefinujmy funkcję, która dzieli prostą sekwencję na próbki

In [ ]:
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# znajdźmy koniec wzoru w sekwencji
		end_ix = i + n_steps
		# sprawdzenie czy nie jesteśmy poza sekwencją
		if end_ix > len(sequence)-1:
			break
		#  zebranie poczatkowych i końcowych części sekwencji
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [ ]:
#sekwencja
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
#liczba kroków czasowych
n_steps = 3

In [ ]:
# podział na próbki
X, y = split_sequence(raw_seq, n_steps)
# pętla pritnująca próbki dla każdego kroku czasowego i przediału
for i in range(len(X)):
	print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


In [ ]:
# zmieniamy kształt wektora z [samples, timesteps] do [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
print(X.shape)

(6, 3, 1)


In [ ]:
model = Sequential()
model.add(SimpleRNN(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:

model.fit(X, y, epochs=200, verbose=0)
# dokonajmy predykcji z uzyciem sekwencji [70, 80, 90]
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
y_predict = model.predict(x_input, verbose=0)
print(y_predict)

[[105.72211]]


Otrzymana wartośc to prawie 110, oczekiwana to 100. Jak widać zwykły RNN nie radzi sobie najlepiej z taką predykcją.
Dokonajmy pewnej zmiany i użyjmy komórek LSTM.
Zdefiniujmy nowy model

In [ ]:
model_LSTM = Sequential()
model_LSTM.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model_LSTM.add(Dense(1))
model_LSTM.compile(optimizer='adam', loss='mse')

Wykonajmy predykcje z użyciem nowego modelu:

In [ ]:
model_LSTM.fit(X, y, epochs=200, verbose=0)
# dokonajmy ponownej predykcji z uzyciem sekwencji [70, 80, 90]
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
y_predict = model_LSTM.predict(x_input, verbose=0)
print(y_predict)

[[102.356316]]


Jak widać wartość otrzymana w wyniku użycia LSTM jest znacznie bliższa faktycznego stanu niż podczas korzystania z prostego RNN
Sprawdźmy jeszcze jak wynik będzie wyglądał w przypadku użycia warstwy GRU

In [ ]:
model_GRU = Sequential()
model_GRU.add(GRU(50, activation='relu', input_shape=(n_steps, n_features)))
model_GRU.add(Dense(1))
model_GRU.compile(optimizer='adam', loss='mse')

Teraz dokonajmy predykcji, ponownie ta sama sekwencja wejściowa:

In [ ]:
model_GRU.fit(X, y, epochs=200, verbose=0)
#Predykcja z użyciem sekwencji [70, 80, 90]
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
predict_y = model_GRU.predict(x_input, verbose=0)
print(predict_y )

[[102.25266]]


Wyniki dla LSTM i GRU są bardzo zbliżone, jak widać nowoczesne wersje sieci RNN spisują się lepiej niż klasyczne podejście

Zachęcam do eksperymentowania z notatnikiem, prób zmiany sekwencji, korków czasowych, liczby neuronów, funkcji aktywacji itp.

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>